In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/ctpn

/content/drive/.shortcut-targets-by-id/1QugxYUpNnNSxf-TEL_ZaWp6mW79dKh92/ctpn


In [ ]:
!pip install tensorflow==1.13.2

     |████████████████████████████████| 92.7 MB 34 kB/s 
     |████████████████████████████████| 50 kB 9.3 MB/s 
     |████████████████████████████████| 367 kB 75.3 MB/s 
     |████████████████████████████████| 3.2 MB 62.8 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.7.0
    Uninstalling tensorflow-2.7.0:
      Successfully uninstalled tensorflow-2.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.7 requires tensorflow>=2.0.0, but you 

In [ ]:
import os, sys
import numpy as np
import pandas as pd
import cv2
import time
import math
from imutils.object_detection import non_max_suppression
from google.colab.patches import cv2_imshow
from operator import itemgetter
from ast import literal_eval
from tqdm import tqdm

In [ ]:
from __future__ import print_function

import glob
import os
import shutil
import sys

import cv2
import numpy as np
import tensorflow as tf
from tensorflow.python.platform import gfile
import time

sys.path.append(os.getcwd())
from lib.fast_rcnn.config import cfg, cfg_from_file
from lib.fast_rcnn.test import _get_blobs
from lib.text_connector.detectors import TextDetector
from lib.text_connector.text_connect_cfg import Config as TextLineCfg
from lib.rpn_msr.proposal_layer_tf import proposal_layer

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

# Truth Values CSV

In [ ]:
def initializeDF():
  # Load the truth values CSV for test set
  test_values = "/content/drive/MyDrive/Datasets/RoadTextVideos/Truthvalues.csv"
  df = pd.read_csv(test_values)
  df["pred_Bbox"] = None
  return df

In [ ]:
df = initializeDF()

# CTPN IMPLEMENTATION

In [ ]:
def resize_im(im, scale, max_scale=None):
    f = float(scale) / min(im.shape[0], im.shape[1])
    if max_scale != None and f * max(im.shape[0], im.shape[1]) > max_scale:
        f = float(max_scale) / max(im.shape[0], im.shape[1])
    return cv2.resize(im, None, None, fx=f, fy=f, interpolation=cv2.INTER_LINEAR), f

In [ ]:
def draw_boxes(img, image_name, boxes, scale, video):
    base_name = image_name.split('/')[-1]
    p_bboxes = []
    filename = OUTPUT_DIR + video + '/' + '{}_{}.txt'.format(video,base_name.split('.')[0])
    #print(filename)
    with open(filename, 'w') as f:
        for box in boxes:
            if np.linalg.norm(box[0] - box[1]) < 5 or np.linalg.norm(box[3] - box[0]) < 5:
                continue
            if box[8] >= 0.9:
                color = (0, 255, 0)
            elif box[8] >= 0.8:
                color = (255, 0, 0)
            cv2.line(img, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), color, 2)
            cv2.line(img, (int(box[0]), int(box[1])), (int(box[4]), int(box[5])), color, 2)
            cv2.line(img, (int(box[6]), int(box[7])), (int(box[2]), int(box[3])), color, 2)
            cv2.line(img, (int(box[4]), int(box[5])), (int(box[6]), int(box[7])), color, 2)

            min_x = min(int(box[0] / scale), int(box[2] / scale), int(box[4] / scale), int(box[6] / scale))
            min_y = min(int(box[1] / scale), int(box[3] / scale), int(box[5] / scale), int(box[7] / scale))
            max_x = max(int(box[0] / scale), int(box[2] / scale), int(box[4] / scale), int(box[6] / scale))
            max_y = max(int(box[1] / scale), int(box[3] / scale), int(box[5] / scale), int(box[7] / scale))

            line = ','.join([str(min_x), str(min_y), str(max_x), str(max_y)]) + '\r\n'
            f.write(line)
            p_bboxes.append({'x1':min_x,'x2':max_x,'y1':min_y,'y2':max_y})

    img = cv2.resize(img, None, None, fx=1.0 / scale, fy=1.0 / scale, interpolation=cv2.INTER_LINEAR)
    cv2.imwrite(os.path.join(OUTPUT_DIR,video,base_name), img)
    return p_bboxes

In [ ]:
#Initialize once

cfg_from_file('ctpn/text.yml')
# init session
config = tf.ConfigProto(allow_soft_placement=True)
sess = tf.Session(config=config)
with gfile.FastGFile('data/ctpn.pb', 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    sess.graph.as_default()
    tf.import_graph_def(graph_def, name='')
sess.run(tf.global_variables_initializer())

input_img = sess.graph.get_tensor_by_name('Placeholder:0')
output_cls_prob = sess.graph.get_tensor_by_name('Reshape_2:0')
output_box_pred = sess.graph.get_tensor_by_name('rpn_bbox_pred/Reshape_1:0')

Instructions for updating:
Use tf.gfile.GFile.


In [ ]:
def detectCTPN(video):
  # Make output folder for video
  if (os.path.exists(OUTPUT_DIR + video + "/")==False):
    os.makedirs(OUTPUT_DIR + video + "/")

  im_names = glob.glob(os.path.join(cfg.DATA_DIR, video, '*.jpg'))

  for im_name in im_names:
    # Skip if file already exists in output dir
    im_output_path = OUTPUT_DIR + video + "/" + im_name.split('/')[-1]
    if (os.path.exists(im_output_path)):
      #print("skipping " + im_name)
      continue
    #print(('Demo for {:s}'.format(im_name)))
    start = time.time()
    img = cv2.imread(im_name)
    img, scale = resize_im(img, scale=TextLineCfg.SCALE, max_scale=TextLineCfg.MAX_SCALE)
    blobs, im_scales = _get_blobs(img, None)
    if cfg.TEST.HAS_RPN:
      im_blob = blobs['data']
      blobs['im_info'] = np.array([[im_blob.shape[1], im_blob.shape[2], im_scales[0]]],dtype=np.float32)
      cls_prob, box_pred = sess.run([output_cls_prob, output_box_pred], feed_dict={input_img: blobs['data']})
      rois, _ = proposal_layer(cls_prob, box_pred, blobs['im_info'], 'TEST', anchor_scales=cfg.ANCHOR_SCALES)

      scores = rois[:, 0]
      boxes = rois[:, 1:5] / im_scales[0]
      textdetector = TextDetector()
      boxes = textdetector.detect(boxes, scores[:, np.newaxis], img.shape[:2])
      costTime = (time.time()-start)
      #print("cost time: {:.2f}s".format(costTime))

      p_bboxes = draw_boxes(img, im_name, boxes, scale, video)

# RUN CTPN HERE

In [ ]:
OUTPUT_DIR = "/content/drive/MyDrive/Datasets/RoadTextVideos/testOutputCTPN/"
INPUT_DIR = "/content/drive/MyDrive/Datasets/RoadTextVideos/test/"

In [ ]:
# Change this for 700,800 and 900 folder
cfg.DATA_DIR = '/content/drive/MyDrive/Datasets/RoadTextVideos/test/900'

In [ ]:
# Run for videos
for video in tqdm(range(951,953)):
  detectCTPN(str(video))

100%|██████████| 2/2 [1:31:55<00:00, 2757.59s/it]


# EVALUATION IMPLEMENTATION

In [ ]:
def IOUScore(boxA,boxB):
    x1,x2,y1,y2 = boxA['x1'], boxA['x2'], boxA['y1'], boxA['y2']
    x3,x4,y3,y4 = boxB['x1'], boxB['x2'], boxB['y1'], boxB['y2']

    dx = min(x2, x4) - max(x1, x3)
    dy = min(y2, y4) - max(y1,y3)
    if (dx>=0) and (dy>=0):
        area_inter = dx*dy

    else:
        area_inter = 0
    # print(area_inter)

    width_box1 = abs(x2-x1)
    height_box1 = abs(y2-y1)
    width_box2 = abs(x4-x3)
    height_box2 = abs(y4-y3)

    area_box1 = width_box1*height_box1
    area_box2 = width_box2*height_box2

    # print(area_box1)
    # print(area_box2)
    area_union = area_box1 + area_box2 - area_inter

    iou = area_inter/area_union
    return iou

In [ ]:
def getBboxVal(vid,frame):

  truthValue = df.loc[(df['videoName'] == int(vid)) & (df['index'] == int(frame)),"values"].values[0]
  if(isinstance(truthValue,str) is False):
    if(math.isnan(truthValue)):
      truthListSorted=[]
  else:
    truthListUnsorted = literal_eval(truthValue)
    # Sort List w.r.t x1
    truthListSorted = sorted(truthListUnsorted, key = itemgetter('x1'))

  return truthListSorted

In [ ]:
def getIOUScoreFromBbox(p_bboxes,gt_bboxes,threshold):
  # Search for match between Ground Truth and predicted bbox
  tp=0
  fp=0
  tn=0
  for p_box in p_bboxes:
    for gt_box in gt_bboxes:
      iou = IOUScore(p_box,gt_box)
      if (iou>=threshold):
        #True Positive
        # print("true positive " + str(iou))
        tp+=1
        break
  fp = len(p_bboxes) - tp
  fn = len(gt_bboxes) - tp

  return tp,fp,fn

In [ ]:
def evaluateCTPN(threshold):
  #Evaluating CTPN Model for the test set
  TP=0
  FP=0
  FN=0

  # Find all txt files under the output directory (including subdirectories)
  file_names = glob.glob(OUTPUT_DIR + '*/*.txt')

  for file in file_names:
    # print("FILENAME "+ file)
    base_name = file.split('/')[-1]

    # Get video and frame number for ground truth comparison
    vid = base_name.split('_')[0]
    frame = base_name.split('_')[1].split('.')[0]
    # print("vid " + vid)
    # print("frame " + frame)

    # Read the file and get the predicted bbox values
    p_bboxes = []
    Lines = open(file, 'r').readlines() # Multiple bounding boxes in one img
    for line in Lines:
      x1,y1,x2,y2 = line.split(',')
      p_bboxes.append({'x1':int(x1),'x2':int(x2),'y1':int(y1),'y2':int(y2)})
    # print(p_bboxes)

    #Get ground truth bbox values
    gt_bboxes = getBboxVal(vid,frame)
    # print(gt_bboxes)

    # Compare predicted bbox values to ground truth values
    tp,fp,fn = getIOUScoreFromBbox(p_bboxes,gt_bboxes,threshold)

    TP+=tp
    FP+=fp
    FN+=fn

  return TP,FP,FN

# RUN EVALUATION HERE

In [ ]:
TP,FP,FN = evaluateCTPN(0.5)

In [ ]:
Precision = TP/(TP+FP)
Recall = TP/(TP+FN)
print("Precision ",Precision,"\nRecall ",Recall)

Precision  0.10909090909090909 
Recall  0.1323529411764706
